In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time
import json

# Scrape car listings from Avito.ma

In [2]:
BASE_URL = "https://www.avito.ma"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_car_links(pages=5):
    car_links = []
    
    for page in range(0, pages ):
        # Construction des URLs 
        if page == 1:
            url = f"{BASE_URL}/fr/maroc/voiture"
        else:
            url = f"{BASE_URL}/fr/maroc/voiture?o={page}"
        
        try:
            response = requests.get(url, headers=HEADERS)
            response.raise_for_status() 
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Sélecteur précis avec vérification des liens de voitures
            listings = soup.select('a.sc-1jge648-0[href*="/fr/"][href*="voitures"]')
            
            for link in listings:
                full_url = urljoin(BASE_URL, link['href'])
                if full_url not in car_links:
                    car_links.append(full_url)
            
            print(f" Page {page + 1} traitée - {len(listings)} nouvelles annonces")
            time.sleep(3) 
            
        except Exception as e:
            print(f" Erreur page {page}: {str(e)}")
            continue
    
    print(f"\nTotal des annonces uniques trouvées : {len(car_links)}")
    return car_links

# Extracting detailed information from individual car listing pages on Avito.ma

In [3]:
def scrape_car_details(url):
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        script = soup.find('script', id="__NEXT_DATA__")
    
        if not script or not script.string:
            print("Script non trouvé ou vide.")
            return None
            
        try:
            scre = json.loads(script.string)
            
        
            listing = scre["props"]["pageProps"]["initialReduxState"]["ad"]["view"]
            details = {
                        "titre": listing.get("adInfo", {}).get("subject", ""),
                        "prix": listing.get("adInfo", {}).get("price", {}).get("value", "NULL"),
                        "ville": listing.get("adInfo", {}).get("location", {}).get("city", {}).get("name", "NULL"),
                        "description": listing.get("adInfo", {}).get("description", "NULL"),
                        "type": listing.get("adInfo", {}).get("type", {}).get("label", "NULL"),
                        "category": listing.get("adInfo", {}).get("category", {}).get("name", "NULL"),
                        "phone": listing.get("adInfo", {}).get("phone", "NULL"),
                        "seller_name": listing.get("adInfo", {}).get("seller", {}).get("name", "NULL"),
                        "seller_address": listing.get("adInfo", {}).get("seller", {}).get("address", "NULL"),
                        "seller_img": listing.get("adInfo", {}).get("seller", {}).get("img", "NULL"),
                        "seller_uuid": listing.get("adInfo", {}).get("seller", {}).get("uuid", "NULL"),
                        "images": [image.get("paths", {}).get("standard", "NULL") for image in listing.get("adInfo", {}).get("images", [])],
                        "url": listing.get("adInfo", {}).get("friendlyUrl", {}).get("url", "NULL")
                    }
            return details
            
        except (json.JSONDecodeError, KeyError) as json_err:
                print("Erreur lors du parsing JSON ou clé manquante:", json_err)
                return None

    except requests.RequestException as e:
        print(f"Erreur de requête sur {url[:50]}...: {str(e)}")
        return None


#  Main scraping process. 

In [4]:
if __name__ == "__main__":
    print("Debut du scraping Avito.ma...")
    
# Étape 1 : Récupération des liens
    print("Collecte des liens d'annonces...")
    car_links = get_car_links(pages=30) 
    print(f"{len(car_links)} lien(s) trouvé(s).")

# Étape 2 : Scraping des détails de chaque annonce
    print(f"Debut du scraping des {len(car_links)} annonces...")
    results = []

    for i, link in enumerate(car_links, 1):
        print(f"\n Traitement {i}/{len(car_links)}: {link[:60]}...")
        try:
            car_data = scrape_car_details(link)
            print('----------- : ',car_data)
            if car_data:
                results.append(car_data)
            else:
                print("ERREUR")
        except Exception as e:
            print(f"\n Erreur inattendue sur {link[:60]}...: {str(e)}")
        
        time.sleep(4) 

# Étape 3 : Sauvegarde des résultats en JSON
    if results:
        with open('avito_voitures_20242.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    else:
        print("\n Aucune donnée récupérée.")


Debut du scraping Avito.ma...
Collecte des liens d'annonces...
 Page 1 traitée - 31 nouvelles annonces
 Page 2 traitée - 31 nouvelles annonces
 Page 3 traitée - 32 nouvelles annonces
 Page 4 traitée - 35 nouvelles annonces
 Page 5 traitée - 35 nouvelles annonces
 Page 6 traitée - 33 nouvelles annonces
 Page 7 traitée - 32 nouvelles annonces
 Page 8 traitée - 32 nouvelles annonces
 Page 9 traitée - 35 nouvelles annonces
 Page 10 traitée - 30 nouvelles annonces
 Page 11 traitée - 33 nouvelles annonces
 Page 12 traitée - 33 nouvelles annonces
 Page 13 traitée - 35 nouvelles annonces
 Page 14 traitée - 34 nouvelles annonces
 Page 15 traitée - 31 nouvelles annonces
 Page 16 traitée - 33 nouvelles annonces
 Page 17 traitée - 32 nouvelles annonces
 Page 18 traitée - 34 nouvelles annonces
 Page 19 traitée - 32 nouvelles annonces
 Page 20 traitée - 32 nouvelles annonces
 Page 21 traitée - 33 nouvelles annonces
 Page 22 traitée - 31 nouvelles annonces
 Page 23 traitée - 33 nouvelles annonces
 Pa

# Charger les données depuis le fichier JSON : overwrite the excel file

In [6]:
with open('avito_voitures_20242.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convertir en DataFrame
df = pd.DataFrame(data)

# Enregistrer en Excel
df.to_excel('voitures_avito.xlsx', index=False, engine='openpyxl')

print("Fichier Excel créé avec succès!")

Fichier Excel créé avec succès!
